<a href="https://colab.research.google.com/github/pockerman/hidden_markov_modeling/blob/master/stories/story_15.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Story 15


In [1]:
from itertools import cycle, islice
import numpy as np

In [2]:
from sklearn import mixture
from sklearn import metrics

In [7]:
from scipy.stats import kde
from scipy import linalg
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns

In [ ]:
from helpers import read_configuration_file
from train import main
from train import load_regions
from hmm_helpers import build_hmm
from helpers import WindowType

In [8]:
sns.set(color_codes=True)

In [9]:
def load_data_file(filename):

    with open(filename) as file:
        context = file.read()
        size = len(context)
        arraystr= context[1:size-1]
        arraystr = arraystr.split(',')
        region_means = [float(item) for item in arraystr]
        return region_means

In [15]:
def make_data_array(wga_mu, no_wga_mu, gc, use_ratio, use_gc):
    data = []
    
    if use_ratio and use_gc:
        for no_wga_val, wga_val,gc_val in zip(no_wga_mu, wga_mu, gc):
            data.append([no_wga_val, wga_val, (wga_val + 1)/(no_wga_val + 1), gc_val])
    elif use_ratio:
        for no_wga, wga  in zip(no_wga_mu, wga_mu):
            data.append([no_wga, wga,  (wga + 1)/(no_wga + 1)])
    elif use_gc:
        
        for no_wga_val, wga_val , gc_val in zip(no_wga_mu, wga_mu,  gc):
            data.append([no_wga_val, wga_val, gc_val])
    else:
        
        for no_wga, wga  in zip(no_wga_mu, wga_mu):
            data.append([no_wga, wga ])
        
    return data

In [ ]:
def gmm_clustering(clusters, data, cov_type, tol, max_itrs, n_init, no_wga_mu, wga_mu):
    
    #for nclusters in clusters:
    
        print("Number of clusters ", clusters)
        gmm = mixture.GaussianMixture(n_components=clusters,
                                      covariance_type=cov_type,
                                      tol=tol, max_iter=max_itrs,
                                     n_init=n_init)
        gmm.fit(data)
        print("Converged: ", gmm.converged_)
        print("BIC: ", gmm.bic(data))
        labels = gmm.predict(data)

        #colors = np.array(list(islice(cycle(['#377eb8', '#ff7f00', '#4daf4a',
        #                                     '#f781bf', '#a65628', '#984ea3',
        #                                     '#999999', '#e41a1c', '#dede00']),
        #                                  int(max(labels) + 1))))
        
        colors = np.array(['#377eb8', '#ff7f00', '#4daf4a',
                            '#f781bf', '#a65628'])

        # add black color for outliers (if any)
        colors = np.append(colors, ["#000000"])

        plt.scatter(no_wga_mu, wga_mu,  color=colors[labels])
        
        kernel= kde.gaussian_kde(np.vstack([no_wga_mu, wga_mu]))
        xi, yi = np.mgrid[min(no_wga_mu):max(no_wga_mu):nbins*1j, 
                          min(wga_mu):max(wga_mu):nbins*1j]
        zi = kernel(np.vstack([xi.flatten(), yi.flatten()]))
        #plt.pcolormesh(xi, yi, zi.reshape(xi.shape), cmap='Blues')
        plt.contour(xi, yi, zi.reshape(xi.shape), 24 )
        
        plt.xlabel("NO-WGA ")
        plt.ylabel("WGA")
        plt.show()
        
        return gmm

In [10]:
# change here accordingly the paths


wga_mean_tuf_I_file = "/home/a/ag568/wga_windows_mean_0_TUF_DETAIL_I.txt"
no_wga_mean_tuf_I_file = "/home/a/ag568/no_wga_windows_mean_0_TUF_DETAIL_I.txt"
gc_tuf_I_file = "/home/a/ag568/windows_gc_0_TUF_DETAIL_I.txt"

#wga_mean_tuf_II_file = "/home/a/ag568/wga_windows_mean_0_TUF_DETAIL_II.txt"
#no_wga_mean_tuf_II_file = "/home/a/ag568/no_wga_windows_mean_0_TUF_DETAIL_II.txt"
#gc_tuf_II_file = "/home/a/ag568/windows_gc_0_TUF_DETAIL_II.txt"

wga_mean_single_copy_deletion_file = "/home/a/ag568/wga_windows_mean_0_SINGLE_COPY_DELETION.txt"
no_wga_mean_single_copy_deletion_file = "/home/a/ag568/no_wga_windows_mean_0_SINGLE_COPY_DELETION.txt"
gc_single_copy_deletion_file = "/home/a/ag568/windows_gc_0_SINGLE_COPY_DELETION.txt"

wga_mean_duplication_file = "/home/a/ag568/wga_windows_mean_0_DUPLICATION.txt"
no_wga_mean_duplication_file = "/home/a/ag568/no_wga_windows_mean_0_DUPLICATION.txt"
gc_duplication_file = "/home/a/ag568/windows_gc_0_DUPLICATION.txt"

wga_mean_delete_file = "/home/a/ag568/wga_windows_mean_0_DELETE.txt"
no_wga_mean_delete_file = "/home/a/ag568/no_wga_windows_mean_0_DELETE.txt"
gc_delete_file = "/home/a/ag568/windows_gc_0_DELETE.txt"

In [11]:
wga_mu_tuf_I = load_data_file(filename=wga_mean_tuf_I_file)
no_wga_mu_tuf_I = load_data_file(filename=no_wga_mean_tuf_I_file)
gc_tuf_I = load_data_file(filename=gc_tuf_I_file)

FileNotFoundError: [Errno 2] No such file or directory: '/home/a/ag568/wga_windows_mean_0_TUF_DETAIL_I.txt'

In [12]:
wga_mu_single_copy_deletion = load_data_file(filename=wga_mean_single_copy_deletion_file)
no_wga_mu_single_copy_deletion = load_data_file(filename=no_wga_mean_single_copy_deletion_file)
gc_single_copy_deletion = load_data_file(filename=gc_single_copy_deletion_file)

FileNotFoundError: [Errno 2] No such file or directory: '/home/a/ag568/wga_windows_mean_0_SINGLE_COPY_DELETION.txt'

In [13]:
wga_mu_duplication = load_data_file(filename=wga_mean_duplication_file)
no_wga_mu_duplication = load_data_file(filename=no_wga_mean_duplication_file)
gc_duplication = load_data_file(filename=gc_duplication_file)

FileNotFoundError: [Errno 2] No such file or directory: '/home/a/ag568/wga_windows_mean_0_DUPLICATION.txt'

In [14]:
wga_mu_delete = load_data_file(filename=wga_mean_delete_file)
no_wga_mu_delete = load_data_file(filename=no_wga_mean_delete_file)
gc_delete = load_data_file(filename=gc_delete_file)

FileNotFoundError: [Errno 2] No such file or directory: '/home/a/ag568/wga_windows_mean_0_DELETE.txt'

In [16]:
# mix the data

# WGA sample
wga_mu = []
wga_mu.extend(wga_mu_tuf_I)
#wga_mu.extend(wga_mu_tuf_II)
wga_mu.extend(wga_mu_single_copy_deletion)
wga_mu.extend(wga_mu_duplication)
wga_mu.extend(wga_mu_delete)


# NO-WGA sample
no_wga_mu = []
no_wga_mu.extend(no_wga_mu_tuf_I)
#no_wga_mu.extend(no_wga_mu_tuf_II)
no_wga_mu.extend(no_wga_mu_single_copy_deletion)
no_wga_mu.extend(no_wga_mu_duplication)
no_wga_mu.extend(no_wga_mu_delete)


# GC
gc = []
gc.extend(gc_tuf_I)
#gc.extend(gc_tuf_II)
gc.extend(gc_single_copy_deletion)
gc.extend(gc_duplication)
gc.extend(gc_delete)


NameError: name 'wga_mu_tuf_I' is not defined

## Cluster the reference data

In [ ]:
data = make_data_array(wga_mu=wga_mu, 
                       no_wga_mu=no_wga_mu, gc=None, 
                       use_ratio=False, use_gc=False)

data = np.array(data)

assert data.shape == (len(wga_mu), 2)

In [17]:
gmm = gmm_clustering(clusters=5, data=data, tol=1.0e-5, 
                     cov_type='diag',
                     max_itrs=300, n_init=1,
                     no_wga_mu=no_wga_mu, wga_mu=wga_mu)

NameError: name 'gmm_clustering' is not defined

### Cluster for TUF

Simply use the ref TUF I region to extract the distribution.

In [ ]:
tuf_wga_mu = []
tuf_wga_mu.extend(wga_mu_tuf_I)

tuf_no_wga_mu = []
tuf_no_wga_mu.extend(no_wga_mu_tuf_I)

In [18]:
data = make_data_array(wga_mu=tuf_wga_mu, 
                       no_wga_mu=tuf_no_wga_mu, gc=None, 
                       use_ratio=True, use_gc=False)

data = np.array(data)

assert data.shape == (len(wga_mu), 3)

NameError: name 'no_wga_mu' is not defined

In [ ]:
gmm_tuf = gmm_clustering(clusters=3, data=data, tol=1.0e-5, cov_type='diag',
                         max_itrs=300, n_init=1,
                         no_wga_mu=no_wga_mu, wga_mu=wga_mu)

## Form the distributions

## Apply HMM

In [21]:
# load the configuration
configuration = read_configuration_file("../config.json")

#configuration["clusters"] = {}
#configuration["HMM"] = {"states":{}, "transitions" }

In [20]:

clusters_config = configuration["clusters"]

NameError: name 'configuration' is not defined

In [22]:
hmm_states = configuration["HMM"]["states"]

KeyError: 'HMM'

In [23]:
hmm_transitions =  configuration["HMM"]["transitions"]

KeyError: 'HMM'

### Train HMM

In [24]:
# now we can train
hmm, regions_list = main(configuration=configuration)

NameError: name 'main' is not defined

In [25]:
# visualize the model we just trained
plt.figure( figsize=(20,18) )
hmm.plot()
plt.show()

NameError: name 'hmm' is not defined

<Figure size 1440x1296 with 0 Axes>

In [26]:
# load a sequence including the gaps
sequence = regions[0].get_region_as_rd_mean_sequences_with_windows(size=None,
                                                                   window_type=WindowType.from_string(hmm_config["train_windowtype"]),
                                                                   n_seqs=hmm_config["train_n_sequences_per_source"],
                                                                   exclude_gaps=False)

NameError: name 'regions' is not defined

In [ ]:
observations = []
for i in range(len(sequence)):
    observations.append(sequence[i][0])

    print("Sequence length: ",len(sequence))

    time_start = time.perf_counter()
    viterbi_path = hmm.viterbi(observations)
    time_end = time.perf_counter()
    print("Done. Execution time"
          " {0} secs".format(time_end - time_start))
    print("Log-probability of ML Viterbi path: ", viterbi_path[0])


    if viterbi_path[1] is not None:
        print("Viterbi path length: ", len(viterbi_path[1]))

        filename="viterbi_path.txt"
        counter = 0
        with open(filename, 'w') as f:
            f.write(str(len(viterbi_path[1])-1) + "\n")
            for item in range(len(sequence)):

                if sequence[item][0] == (-999.0, -999.0):
                    counter += 1

                f.write(str(item)+ ":" + str(sequence[item][1]) + ":" + str(sequence[item][0]) + ":" + viterbi_path[1][item+1][1].name + "\n")
                #print("sequnce item: {0} state {1}".format(sequence[item], viterbi_path[1][item+1][1].name))
        print("There should be {0} gaps".format(counter))
    else:
        print("Viterbi path is impossible for the given sequence")